In [1]:
import numpy as np
import adi
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

In [2]:
sdr = adi.Pluto()
sdr.rx_lo = int(2.4385e9)
sdr.rx_rf_bandwidth = int(56e6)
sdr.sample_rate = int(40e6)
sdr.rx_buffer_size = 1024

In [3]:
sdr

Pluto(uri="") object "PlutoSDR" with following key properties:

rx_lo:                   2438.5       MHz, Carrier frequency of RX path
rx_hardwaregain_chan0    16           dB, Gain applied to RX path. Only applicable when gain_control_mode is set to 'manual'
rx_rf_bandwidth:         56.0         MHz, Bandwidth of front-end analog filter of RX path
gain_control_mode_chan0: slow_attack  Receive path AGC Options: slow_attack, fast_attack, manual

tx_lo:                   2450.0       MHz, Carrier frequency of TX path
tx_hardwaregain_chan0:   -10          dB, Attenuation applied to TX path
tx_rf_bandwidth:         18.0         MHz, Bandwidth of front-end analog filter of TX path
tx_cyclic_buffer:        0            Toggles cyclic buffer

filter:                  [128.0, 2.0, 128.0, 2.0] FIR filter file
sample_rate:             40.0         MSPS, Sample rate RX and TX paths
loopback:                0            0=Disabled, 1=Digital, 2=RF


In [4]:
print("RX Buffer Size", sdr.rx_buffer_size)
print("RX Gain", sdr.rx_hardwaregain_chan0)
print("TX Gain", sdr.tx_hardwaregain_chan0)

RX Buffer Size 1024
RX Gain 14
TX Gain -10


In [5]:
def FFT(samples, buf, samprate, lo):
    fft = np.fft.fftshift(np.fft.fft(samples)) / buf
    freqLabels = np.fft.fftshift(np.fft.fftfreq(buf, 1/samprate)) + lo
    fft_mag_dB = 10*np.log10(np.abs(fft))
#     fft_mag = np.abs(fft)
    return fft_mag_dB, freqLabels

def MA(data):
    N = 10
    w = np.ones(N) / N
    ma = np.convolve(data, w, mode='same')
    return(ma)

def findSigPts(boolarray):
    a = boolarray^np.concatenate(([False],boolarray[:-1]))
    return(abs(a))

def findBWs(sigpts):
    freqs = np.zeros([len(sigpts)-1,2])
    freqs[:,0] = sigpts[1:]
    freqs[:,1] = sigpts[:-1]
    BWs = (sigpts[1:]-sigpts[:-1])/1e6
    return BWs, freqs

def findSigSec(boolarray):
    SigSections = []
    for i in range(1,len(boolarray)):
        if boolarray[i-1]:
            if i == 1:
                SigSections.append(1)
            if not boolarray[i]:
                SigSections.append(0)
        elif not boolarray[i-1]:
            if boolarray[i]:
                SigSections.append(1)

    return np.array(SigSections[:-1]).astype(np.bool)

In [ ]:
from time import sleep

while (True):
    ct = 0
    guess = 0
    for i in range(500):

        droneFreqData, droneFreqLabels = FFT(sdr.rx(), sdr.rx_buffer_size, sdr.sample_rate, sdr.rx_lo)

        freq_ndb = 10**(droneFreqData/10)
        freq_ma = MA(droneFreqData)
        freq_ma_ndb = MA(freq_ndb)
        ma_thresh = freq_ndb.mean()*1.6

        if (np.sum(freq_ma_ndb > ma_thresh) > 0):
            BWs, freqRgs= findBWs(droneFreqLabels[findSigPts(freq_ma_ndb > ma_thresh)])
            sigSec = findSigSec(freq_ma_ndb > ma_thresh)
            sigBWs = BWs[sigSec][(BWs[sigSec] > 9) & (BWs[sigSec] < 12)]
            drone_sig_range = freqRgs[sigSec][(BWs[sigSec] > 9) & (BWs[sigSec] < 12)]

        if sigBWs.size == 1:
            ct += 1
            guess += (drone_sig_range[0,0]-drone_sig_range[0,1])/2 + drone_sig_range[0,1]
    #         clear_output(wait=True)
    #         print(f"Drone BW: {sigBWs}" )
    #         print(f"Drone Freq: {drone_sig_range}")
    #         print(f"Drone Center Freq: {(drone_sig_range[0,0]-drone_sig_range[0,1])/2+sdr.rx_lo}")

    clear_output(wait=True)
    if ct > 0:
        print(f"Drone Located at {guess/ct}")
    print(f"Number of Guesses: {ct}")
#     sleep(1)

Drone Located at 2439549262.1527777
Number of Guesses: 72
